# Imports

In [50]:
import pandas as pd
import numpy as np

import time
import importlib

import data_processing as dp  # own functions and logic
importlib.reload(dp)  # this makes it so that 
                      # the file with our functions is re-read every time, 
                      # in case we have made modifications

<module 'data_processing' from '/home/andreas/Nextcloud/Dokumente/Uni/Module/3sem-EPFL/ada/Project/ada-2024-project-thedataminions/preprocessing_tests/data_processing.py'>

# Configure path to data files

In [51]:
# define the path to the folder where the YouNiverse dataset is stored here

# when adding your own path, don't remove the existing path, just comment it
# in this way, everyone can quickly uncomment their own path
dataset_root_path = "/media/andreas/Backup Plus/youniverse_dataset/"

# Preprocessing

## Loading the data

In [52]:
# load channel data (no chunks needed, as the file is not very large)
df_channels = pd.read_csv(dataset_root_path + "df_channels_en.tsv.gz", compression="infer", sep="\t")


In [53]:
# load a small part of video and comment data, to try functions on them etc.
# these datasets should not be used for calculations, as they don't contain all the data

# load (first 100000 rows of) video data
df_videos = pd.read_json(dataset_root_path + "yt_metadata_en.jsonl.gz", compression="infer", lines=True, nrows=100000)

# load (first 1000000 rows of) comment data
df_comments = pd.read_csv(dataset_root_path + "youtube_comments.tsv.gz", compression="infer", sep="\t", nrows=1000000)

In [54]:
df_channels.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [ ]:
# definition of "readers", i.e., objects that we can iterate through 
# and always get a chunk of the dataframe in each iteration

def videos_in_chunks(chunksize: int = 100000) -> pd.io.json._json.JsonReader:
    """
    Returns a Json reader which can be iterated through, to get chunks of the (unfiltered) video dataset.

    Args:
        chunksize: number of entries in each chunk

    Returns:
        the Json reader
    """
    return pd.read_json(dataset_root_path + "yt_metadata_en.jsonl.gz", 
                        compression="infer", lines=True, chunksize=chunksize, )
                        # nrows=10000000, )   # uncomment this to only use the first million videos, for testing
                                           # (remove the paranthesis above as well)

def comments_in_chunks(chunksize: int = 1000000) -> pd.io.parsers.readers.TextFileReader:
    """
    Returns a CSV reader which can be iterated through, to get chunks of the (unfiltered) comment dataset.

    Args:
        chunksize: number of entries in each chunk

    Returns:
        the CSV reader
    """
    return pd.read_csv(dataset_root_path + "youtube_comments.tsv.gz", 
                       compression="infer", sep="\t", chunksize=chunksize, )
                       # nrows = 10000000)  # uncomment this to only use the first 10 million comments, for testing
                                            # (remove the paranthesis above as well)

## Checking for NaNs in the dataset

*Note:* This is not complete, **(still a**
# Todo
)

Right now, we are only checking the comments dataset for nans, not the other files

also, we are only relying on pd.isna for finding NaNs.
This function however only counts "None" or "np.NaN" and similar as Nans.
If we for example have empty strings, or zeros in places they should not be, this would not be detected.

So we should do some manual searching as well, eg. df_comments[df_comments.user_id == 0] and similar

In [56]:
# get the entries of the comment dataframe which have a na value in any column
nans = dp.run_simple_function_on_chunks_concat(comments_in_chunks(), 
                                        lambda x: dp.get_na_entries(x, "any", reverse=False),
                                        print_time=(1000000, 8600000000))



Going through chunk 0...
The first 1000000 entries have been processed. 8599000000 left.
1.239 secs per chunk on average. Meaning  177.597 minutes left.
Going through chunk 1...
The first 2000000 entries have been processed. 8598000000 left.
1.133 secs per chunk on average. Meaning  162.369 minutes left.
Going through chunk 2...
The first 3000000 entries have been processed. 8597000000 left.
1.151 secs per chunk on average. Meaning  164.961 minutes left.
Going through chunk 3...
The first 4000000 entries have been processed. 8596000000 left.
1.193 secs per chunk on average. Meaning  170.846 minutes left.
Going through chunk 4...
The first 5000000 entries have been processed. 8595000000 left.
1.185 secs per chunk on average. Meaning  169.757 minutes left.
Going through chunk 5...
The first 6000000 entries have been processed. 8594000000 left.
1.189 secs per chunk on average. Meaning  170.327 minutes left.
Going through chunk 6...
The first 7000000 entries have been processed. 8593000000

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
display(nans)

,author,video_id,likes,replies


In [ ]:
# count the entries of the comment dataframe which have a na value in any column
counted_nans = dp.run_simple_function_on_chunks_concat(comments_in_chunks(), 
                                                lambda x: dp.count_na_entries(x, "any", reverse=False),
                                                print_time=(1000000, 8600000000)).sum(axis=0)



Going through chunk 0...
The first 1000000 entries have been processed. 8599000000 left.
0.730 secs per chunk on average. Meaning  104.669 minutes left.
Going through chunk 1...
The first 2000000 entries have been processed. 8598000000 left.
0.726 secs per chunk on average. Meaning  104.055 minutes left.
Going through chunk 2...
The first 3000000 entries have been processed. 8597000000 left.
0.745 secs per chunk on average. Meaning  106.686 minutes left.
Going through chunk 3...
The first 4000000 entries have been processed. 8596000000 left.
0.736 secs per chunk on average. Meaning  105.499 minutes left.
Going through chunk 4...
The first 5000000 entries have been processed. 8595000000 left.
0.756 secs per chunk on average. Meaning  108.361 minutes left.
Going through chunk 5...
The first 6000000 entries have been processed. 8594000000 left.
0.757 secs per chunk on average. Meaning  108.372 minutes left.
Going through chunk 6...
The first 7000000 entries have been processed. 8593000000

In [ ]:
display(counted_nans)

na rows              0
total rows    10000000
dtype: int64

## Filtering the datasets to our needs

### Filtering videos and channels by category News & Politics

In [ ]:
# filter the video dataframe to only include videos from news and politics category

df_videos_news_pol = dp.run_simple_function_on_chunks_concat(videos_in_chunks(chunksize=100000),
                                                             lambda x: x[x.categories == "News & Politics"], 
                                                             print_time=(100000, 72924794))

Going through chunk 0...
The first 100000 entries have been processed. 72824794 left.
3.634 secs per chunk on average. Meaning  44.105 minutes left.
Going through chunk 1...
The first 200000 entries have been processed. 72724794 left.
3.977 secs per chunk on average. Meaning  48.209 minutes left.
Going through chunk 2...
The first 300000 entries have been processed. 72624794 left.
4.049 secs per chunk on average. Meaning  49.011 minutes left.
Going through chunk 3...
The first 400000 entries have been processed. 72524794 left.
3.995 secs per chunk on average. Meaning  48.284 minutes left.
Going through chunk 4...
The first 500000 entries have been processed. 72424794 left.
3.776 secs per chunk on average. Meaning  45.584 minutes left.
Going through chunk 5...
The first 600000 entries have been processed. 72324794 left.
3.477 secs per chunk on average. Meaning  41.913 minutes left.
Going through chunk 6...
The first 700000 entries have been processed. 72224794 left.
3.283 secs per chunk

In [ ]:
df_channels_news_pol = df_channels[df_channels.category_cc == "News & Politics"]

In [ ]:
display(df_videos_news_pol)

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
1827,News & Politics,UCzWm1-4XF7AHxVUTkHCM1uw,2019-11-17 06:28:42.593675,retrogamer3.com,16.0,dfa8RRkKoa4,9251,25.0,"RetroGamer3,Live Stream,politics,Trump",Retrogamer3 Political Stream,2018-08-23 00:00:00,478.0
7605,News & Politics,UCzWLsxDD373D4tY8kN-0LGQ,2019-11-05 00:42:33.012228,What are the forces at work that have created ...,0.0,_dIIEMvH86k,309,9.0,"NWO,Ebola,Ukraine,Mainstream,Media,Pyschology",Adam Curtis describes the Surkow Strategy of M...,2015-01-04 00:00:00,865.0
18005,News & Politics,UCzVBu6oqlrAix0oq9T2rBFg,2019-11-19 20:40:22.403775,Social Media:\n\nFacebook.com/thebookoflaura\n...,89.0,eWXefhNB2po,707,625.0,"michael jackson,lyrics,music video,court,child...",my thoughts on the michael jackson documentary.,2019-04-24 00:00:00,12780.0
24361,News & Politics,UCzUV5283-l5c0oKRtyenj6Q,2019-11-22 08:47:10.520209,👕 Order your shirts here: https://Teespring.co...,195.0,MBgzne7djFU,378,47027.0,"Funny,Entertainment,Fun,Laughing,Educational,L...",Elizabeth Warren Gets a Big Surprise at the Ai...,2019-10-03 00:00:00,374711.0
24362,News & Politics,UCzUV5283-l5c0oKRtyenj6Q,2019-11-22 08:46:16.481889,👕 Order your shirts here: https://Teespring.co...,114.0,AbH3pJnFgY8,278,36384.0,"Funny,Entertainment,Fun,Laughing,Educational,L...",No More Twitter? 😂,2019-10-02 00:00:00,245617.0
...,...,...,...,...,...,...,...,...,...,...,...,...
999870,News & Politics,UCrUkx0UAxgybbbMvWphd62Q,2019-11-10 14:27:26.687460,The Young Turks recently posted a video entitl...,2.0,Rmq0JmUbt8k,857,25.0,"American Joe,American Joe Show,The Young Turks...",Young Turks Caught Lying and Race Baiting.... ...,2018-11-17 00:00:00,273.0
999871,News & Politics,UCrUkx0UAxgybbbMvWphd62Q,2019-11-10 14:27:27.273595,Patriots I need your help growing the American...,0.0,ts__Orp310M,49,34.0,"American Joe,American Joe Show",President says he will send migrant Children B...,2018-11-15 00:00:00,353.0
999872,News & Politics,UCrUkx0UAxgybbbMvWphd62Q,2019-11-10 14:27:27.847348,Patriots I need your help growing the American...,1.0,bQ3_ZMVpiio,298,6.0,"American Joe,American Joe Show,Michael Avenatt...","Creepy Porn Lawyer, and Woman Beater Michael A...",2018-11-14 00:00:00,76.0
999873,News & Politics,UCrUkx0UAxgybbbMvWphd62Q,2019-11-10 14:27:28.400609,Patriots I need your help growing the American...,2.0,q92A939Nyj8,388,2.0,"American Joe,American Joe Show,Midterm Electio...",Midterm Fallout - How Bad is it For Trump?,2018-11-14 00:00:00,38.0


In [ ]:
display(df_channels_news_pol)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
129,News & Politics,2006-08-26,UCttspZesZIDEwwpVIgoZtWQ,IndiaTV,15177282,139814,199.0,2.0870
133,News & Politics,2012-06-01,UCRWFSbif-RFENbBrSiez1DA,ABP NEWS,16274836,129027,207.0,2.0870
212,News & Politics,2017-03-03,UCmphdqZNmqL72WJ2uyiNw5w,ABP NEWS HINDI,10800000,51298,340.0,2.0870
268,News & Politics,2015-03-23,UCx8Z14PpntdaxCt2hakbQLQ,The Lallantop,9120000,9423,438.0,2.0870
337,News & Politics,2007-06-19,UCIvaYmXn910QMdemBG3v1pQ,Zee News,9280000,102648,549.0,2.0870
...,...,...,...,...,...,...,...,...
135820,News & Politics,2010-08-07,UC5rxiCGcNunIi5zI1hMYLMg,Salman Akhtar,10400,40,962468.0,53.1435
135825,News & Politics,2013-02-01,UCLSEJQ8TWtlEkaytaa4Y7lw,WingsOfChrist,10420,61,962547.0,53.1435
135901,News & Politics,2012-10-19,UCnkG_c5cyemVVsgCDoHiXew,The American Mirror,10500,329,963417.0,53.1435
136231,News & Politics,2017-11-25,UC69lWS7UMbBQc-9yqp4nGjA,Patriotism Show,10320,46,975448.0,53.1435


In [ ]:
# check if all videos we found in news&pol are also pulished by a channel in category news&pol

df_videos_news_pol[np.logical_not(df_videos_news_pol.channel_id.isin(df_channels_news_pol.channel))]



,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
1827,News & Politics,UCzWm1-4XF7AHxVUTkHCM1uw,2019-11-17 06:28:42.593675,retrogamer3.com,16.0,dfa8RRkKoa4,9251,25.0,"RetroGamer3,Live Stream,politics,Trump",Retrogamer3 Political Stream,2018-08-23 00:00:00,478.0
7605,News & Politics,UCzWLsxDD373D4tY8kN-0LGQ,2019-11-05 00:42:33.012228,What are the forces at work that have created ...,0.0,_dIIEMvH86k,309,9.0,"NWO,Ebola,Ukraine,Mainstream,Media,Pyschology",Adam Curtis describes the Surkow Strategy of M...,2015-01-04 00:00:00,865.0
18005,News & Politics,UCzVBu6oqlrAix0oq9T2rBFg,2019-11-19 20:40:22.403775,Social Media:\n\nFacebook.com/thebookoflaura\n...,89.0,eWXefhNB2po,707,625.0,"michael jackson,lyrics,music video,court,child...",my thoughts on the michael jackson documentary.,2019-04-24 00:00:00,12780.0
28840,News & Politics,UCzTmNzBxLEHbpZNOCpUTWbA,2019-11-03 04:38:01.617657,A young man is living a normal life with no ca...,16.0,ck6Yl8TNoWs,1257,452.0,"JoiRida,Cheatham,JoiRidaCheatham,Accepted,Detr...",Accepted - Award Winning Short Film,2013-10-13 00:00:00,27366.0
28860,News & Politics,UCzTmNzBxLEHbpZNOCpUTWbA,2019-11-03 04:38:06.565138,Short Film,1.0,tjUajxZAIZ8,422,15.0,"Joi.Rida,Cheatham,joiridacheatham,dread,loc,up...",JoiRida Twin Visit (Introducing Jive Viper),2010-03-04 00:00:00,987.0
...,...,...,...,...,...,...,...,...,...,...,...,...
970869,News & Politics,UCrXcatz6wlNHjuqgf-tglOA,2019-11-07 00:55:48.241832,"As promised, our Wet Head Challenge using the ...",3.0,lIuK9DGtOx8,321,141.0,"challenge,wet,head,gross,wet head challenge,we...",Gross Smoothie Wet Head Challenge 😕,2016-08-23 00:00:00,8941.0
991815,News & Politics,UCrVnMcE3GIyg2rM4gH34YWg,2019-11-10 10:02:03.075065,More Travel News...\nhttp://www.petergreenberg...,0.0,aSWbywb7SBE,423,1.0,"2008,Travel Inspiration,clinton,Presidential,P...",2008 Presidential Candidates Travel Scorecard,2008-01-25 00:00:00,588.0
998347,News & Politics,UCrV-WEtbXrRIkgWgbXLAvcQ,2019-10-31 15:06:30.119011,© 2012 WMG Webisode by Mutemath from The Blue...,3.0,H8-Al6B_J1g,106,17.0,"mutemath,wbr,INDMUSIC,warner bros records",Mutemath - What Happens Before The Show [Webis...,2006-11-04 00:00:00,3136.0
998349,News & Politics,UCrV-WEtbXrRIkgWgbXLAvcQ,2019-10-31 15:06:31.498209,© 2012 WMG Webisode by Mutemath from Park Wes...,2.0,wYI6dWaEHjk,56,8.0,"INDMUSIC,wbr,mutemath,warner bros records",Mutemath - Built for Destruction [Webisode],2006-11-04 00:00:00,1881.0


**We see that indeed, not all videos in the news and politics category belong to a channel in this category!**
A google search shows that apparently, you don't have to have the same category for all videos, but you set a "default" channel category which will be used for videos if you don't change it manually. Also, you can probably change the default category after a while if you want.

This is the reason why most of the news&pol videos are uploaded by a news&pol channe, but not all.

In the paper about the dataset, the authors say that the channel category is actually the "most frequent category", so I guess the video categories are the most relevant, as they are the true categories.

We could try to verify this, if we want to.

### Filtering the News&Pol videos by a list of channels

In [ ]:
# filter news&pol videos to only include the channels we are interested in

# Note: this selection of channels is still "random" (from my head), the final selection whould be from a source

df_videos_news_pol_manually_selected = df_videos_news_pol[df_videos_news_pol.channel_id.isin(["UCupvZG-5ko_eiXAupbDfxWw",  # CNN
                                                                                              "UCXIJgqnII2ZOINSWNOGFThA",  # Fox News
                                                                                              "UC16niRr50-MSBwiO3YDb3RA",  # BBC News
                                                                                              "UCaXkIU1QidjPwiAYu6GcHjg",  # MSNBC
                                                                                            ])]

In [ ]:
df_videos_news_pol_manually_selected

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count


In [ ]:
# Note: we probably don't need this

# sort news&pol channels by subscriber count according to channel crawler
df_channels_news_pol_sort_subscribers = df_channels_news_pol.sort_values(by="subscribers_cc", ascending=False)

In [ ]:
display(df_channels_news_pol_sort_subscribers)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
133,News & Politics,2012-06-01,UCRWFSbif-RFENbBrSiez1DA,ABP NEWS,16274836,129027,207.0,2.087
129,News & Politics,2006-08-26,UCttspZesZIDEwwpVIgoZtWQ,IndiaTV,15177282,139814,199.0,2.087
212,News & Politics,2017-03-03,UCmphdqZNmqL72WJ2uyiNw5w,ABP NEWS HINDI,10800000,51298,340.0,2.087
337,News & Politics,2007-06-19,UCIvaYmXn910QMdemBG3v1pQ,Zee News,9280000,102648,549.0,2.087
268,News & Politics,2015-03-23,UCx8Z14PpntdaxCt2hakbQLQ,The Lallantop,9120000,9423,438.0,2.087
...,...,...,...,...,...,...,...,...
130581,News & Politics,2008-03-26,UC65jmwvHLoCMBJXnzHrGwiA,The Virginian-Pilot,10200,3037,868834.0,14.148
134955,News & Politics,2009-05-17,UCuZTp4-0xPoGUxdKPLiQE6w,Harold Jackson,10133,1330,932456.0,23.492
133413,News & Politics,2016-03-25,UCWtAa1fyxYxR1gAH-IZtaXg,Trumpennials,10000,64,905989.0,14.780
133426,News & Politics,2018-08-31,UCaUCzPRX1bKUwz7a2aJgIDQ,Missing Persons & My...,10000,31,906002.0,14.699


### Filtering the comments by channels

**Note:** The first function doesn't have to be done on the whole video dataset, as it is done here.
The first function can be run on the dataset df_videos_news_pol_manually_selected, as all channels we could be interested in should already have all their videos included in that dataset; no need to consider any other videos. Therefore, we would also not need chunks here.

The function could probably instead just be 
df_videos_news_pol_manually_selected.loc[df_videos_news_pol_manually_selected.channel_id == <channel id of the desired channel>]

(Consider this a 
# Todo
)

However, the second function probably needs to look like this, as we haven't preprocessed the comment data yet, so it is still too big to be considered all at once, so we need the chunks.

In [ ]:
# filter the video dataset to get only videos from a specific channel (here: just a random channel for testing)

videos_from_channel_test = dp.run_simple_function_on_chunks_concat(
    videos_in_chunks(chunksize=100000),
    lambda x: x.loc[x.channel_id == "UCzWrhkg9eK5I8Bm3HfV-unA"],
    print_time=(100000, 72924794))

# use the filtered video dataset to filter the comment dataset, to get comments on videos from a specific channel

comments_from_channel_test = dp.run_simple_function_on_chunks_concat(
    comments_in_chunks(chunksize=1000000), 
    lambda df: df[df.video_id.isin(videos_from_channel_test.display_id)],
    print_time=(1000000, 8600000000))

Going through chunk 0...
The first 100000 entries have been processed. 72824794 left.
3.304 secs per chunk on average. Meaning  40.097 minutes left.
Going through chunk 1...
The first 200000 entries have been processed. 72724794 left.
2.883 secs per chunk on average. Meaning  34.939 minutes left.
Going through chunk 2...
The first 300000 entries have been processed. 72624794 left.
2.706 secs per chunk on average. Meaning  32.759 minutes left.
Going through chunk 3...
The first 400000 entries have been processed. 72524794 left.
2.722 secs per chunk on average. Meaning  32.897 minutes left.
Going through chunk 4...
The first 500000 entries have been processed. 72424794 left.
2.856 secs per chunk on average. Meaning  34.473 minutes left.
Going through chunk 5...
The first 600000 entries have been processed. 72324794 left.
2.942 secs per chunk on average. Meaning  35.458 minutes left.
Going through chunk 6...
The first 700000 entries have been processed. 72224794 left.
3.024 secs per chunk

In [ ]:
display(comments_from_channel_test.head())

,author,video_id,likes,replies
6868268,453667,3vQK78eUg2A,2,1
7094579,468696,SWZG-ba1qDk,15,18
8912192,594074,hn2zYwqSINY,0,1


# Todo: next step

make a function which takes dataframes such as the above "filtered_comments_test" (df with comment data from videos only from one specific channel),
and returns the number of comments by each user id
( filtered_comments_test.groupby("user_id").agg("sum") )

sort so that only users with x number of comments remain

(flexible function where you give the threshold)


In [ ]:
def get_metadata_commenters(comment_channelX,x):
    metadata_commenters = comment_channelX.groupby('author').agg(number_of_comments=('author', 'size')).reset_index()
    metadata_commenters['number_of_videos']= comment_channelX.groupby('author')['video_id'].nunique().values
    #only users with x numbers of comments remain
    metadata_commenters=metadata_commenters[metadata_commenters['number_of_comments']>=x]
    return metadata_commenters

metadata_commenters= get_metadata_commenters(comments_from_channel_test,0)
display(metadata_commenters)

# Todo after that

create dataset of all comments which are under a video in the news and politics category

use this dataset to get the list of videos under which each of the users we found (above) have made a comment

for each pair of users, calculate "number of videos in common (under which both have commented) / min number of videos both users have commented on"
(example: mila commented on 10 videos, andreas on 100 videos, they have 8 videos they both commented on, so the value we calculate is 8/10 = 0.8)
